# Final Project

#### Team: &emsp;&emsp; Sasi Kanduri &emsp;&emsp; Vikas Mishra &emsp;&emsp; Ashish Kotian

### Converting .mat files to jpg and image arrays
##### The dataset we chose is from the follwoing link which has images in .mat files. We have to take the image arrays out of the .mat constructs and pre-process them.

In [38]:
import os
import h5py
import numpy as np
from PIL import Image
import pandas as pd
import cv2
import SimpleITK as sitk
# List of directories to process
directories = [
    'brainTumorDataPublic_1-766',
    'brainTumorDataPublic_767-1532',
    'brainTumorDataPublic_1533-2298',
    'brainTumorDataPublic_2299-3064'
]

y_labels = []
filename_column = []

image_arrays = []
image_classes = []


# Set the paths for the Mat and Jpg folders
mat_folder = f'./all_files'
jpg_folder = f'./Jpg_images'

# Create the Jpg folder if it doesn't exist
# if not os.path.exists(jpg_folder):
#     os.makedirs(jpg_folder)

# Iterate through files in the Mat folder
for filename in os.listdir(mat_folder):
    # Construct the full file paths
    mat_filepath = os.path.join(mat_folder, filename)
    jpg_filepath = os.path.join(jpg_folder, filename.split(".")[0] + '.jpg')

    # Open the mat file
    with h5py.File(mat_filepath, 'r+') as f:

        cjdata = f['cjdata']
        image = np.array(cjdata.get('image')).astype(np.float64)
        label = cjdata.get('label')[0, 0]
        
        if image.shape[0] == 512:
            
            y_labels.append(label - 1)
            filename_column.append(filename.split(".")[0])

            # Perform image processing
            hi = np.max(image)
            lo = np.min(image)
            image = (((image - lo) / (hi - lo)) * 255).astype(np.uint8)

            image = cv2.resize(image, (128, 128), interpolation=cv2.INTER_AREA)
            image_arrays.append(image)
            image_classes.append(label - 1)

            # cv2.imwrite(jpg_filepath, image)
            # t1_image = sitk.GetImageFromArray(image)
            # sitk.WriteImage(t1_image, jpg_filepath)

    
df = pd.DataFrame({'filename':filename_column, 'label':y_labels})
df.to_csv('final_df.csv', index=False)

### Data pre-processing

In [ ]:
import numpy as np
from numpy import array
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.utils import plot_model, image_dataset_from_directory
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D, Rescaling
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


import tensorflow as tf

In [40]:
df = pd.read_csv("final_df.csv", index_col=False)

In [41]:
df.sort_values(by='filename', inplace=True)

In [42]:
df.dtypes

filename      int64
label       float64
dtype: object

In [43]:
df['label'] = df['label'].astype('int32')
df['label'].unique()

array([0, 2, 1], dtype=int32)

In [44]:
from keras.utils import to_categorical

y_labels = df['label'].to_numpy().tolist()
y_labels


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
image_arrays = np.array(image_arrays)

In [ ]:
image_arrays = image_arrays.reshape(image_arrays.shape[0], 128, 128, 1)

In [ ]:
image_arrays[0]

array([[[0],
        [2],
        [5],
        ...,
        [5],
        [2],
        [0]],

       [[0],
        [2],
        [5],
        ...,
        [6],
        [2],
        [0]],

       [[0],
        [2],
        [4],
        ...,
        [6],
        [3],
        [0]],

       ...,

       [[1],
        [4],
        [6],
        ...,
        [4],
        [3],
        [0]],

       [[1],
        [4],
        [6],
        ...,
        [5],
        [2],
        [0]],

       [[1],
        [3],
        [6],
        ...,
        [4],
        [2],
        [0]]], dtype=uint8)

In [ ]:
image_classes_encoded = tf.keras.utils.to_categorical(image_classes, 3, "float32")
image_classes_encoded

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [ ]:
print(image_arrays.shape)
print(image_classes_encoded.shape)

(3049, 128, 128, 1)
(3049, 3)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(image_arrays, image_classes_encoded, test_size=0.20, random_state=42)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2286, 128, 128, 1)
(763, 128, 128, 1)
(2286, 3)
(763, 3)


### Baseline model from the paper

In [56]:
model = Sequential()

model.add(Input(shape=(128, 128, 1)))
model.add(Rescaling(1./255)) # scale pixels)
#1

model.add(Conv2D(64, kernel_size=(5, 5), padding='same', strides=1,
                 activation='relu')) 

model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.2))

# 2

model.add(Conv2D(64, kernel_size=(2, 2), padding='same', strides=1,
                 activation='relu')) 

model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.2))

# 3

model.add(Conv2D(64, kernel_size=(2, 2), padding='same', strides=1,
                 activation='relu')) 

model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.2))

# 4

model.add(Conv2D(64, kernel_size=(2, 2), padding='same', strides=1,
                 activation='relu')) 

model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.2))

# 5

model.add(Conv2D(64, kernel_size=(2, 2), padding='same', strides=1,
                 activation='relu')) 

model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.2))

# flatten and dense

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 128, 128, 1)       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 128, 128, 64)      1664      
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 64, 64, 64)        0         
 ng2D)                                                           
                                                                 
 dropout_10 (Dropout)        (None, 64, 64, 64)        0         
                                                                 
 conv2d_11 (Conv2D)          (None, 64, 64, 64)        16448     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 32, 32, 64)        0         
 ng2D)                                                

In [58]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [66]:
checkpointer = ModelCheckpoint(filepath="array_model.hdf5", verbose=2, save_best_only=True, monitor='val_loss')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

In [67]:
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test), callbacks=[monitor, checkpointer])

Epoch 1/20


2023-11-22 23:48:24.168162: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


72/72 [==============================] - ETA: 0s - loss: 0.8676 - accuracy: 0.6203
Epoch 1: val_loss improved from inf to 0.72906, saving model to array_model.hdf5
72/72 [==============================] - 7s 57ms/step - loss: 0.8676 - accuracy: 0.6203 - val_loss: 0.7291 - val_accuracy: 0.6606
Epoch 2/20
 3/72 [>.............................] - ETA: 2s - loss: 0.6527 - accuracy: 0.6875

/Users/shashi/Documents/219-ML/mlenv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


71/72 [============================>.] - ETA: 0s - loss: 0.6443 - accuracy: 0.7170
Epoch 2: val_loss improved from 0.72906 to 0.60379, saving model to array_model.hdf5
72/72 [==============================] - 3s 41ms/step - loss: 0.6457 - accuracy: 0.7165 - val_loss: 0.6038 - val_accuracy: 0.7274
Epoch 3/20
71/72 [============================>.] - ETA: 0s - loss: 0.5822 - accuracy: 0.7443
Epoch 3: val_loss improved from 0.60379 to 0.59030, saving model to array_model.hdf5
72/72 [==============================] - 3s 41ms/step - loss: 0.5808 - accuracy: 0.7454 - val_loss: 0.5903 - val_accuracy: 0.7130
Epoch 4/20
71/72 [============================>.] - ETA: 0s - loss: 0.5070 - accuracy: 0.7614
Epoch 4: val_loss did not improve from 0.59030
72/72 [==============================] - 27s 374ms/step - loss: 0.5063 - accuracy: 0.7620 - val_loss: 0.6010 - val_accuracy: 0.7287
Epoch 5/20
71/72 [============================>.] - ETA: 0s - loss: 0.4633 - accuracy: 0.7914
Epoch 5: val_loss did not 

In [68]:
model.load_weights("array_model.hdf5")
pred = model.predict(x_test)
pred

24/24 [==============================] - 0s 10ms/step


array([[1.0274457e-01, 8.9692158e-01, 3.3381802e-04],
       [9.3609728e-03, 9.9038213e-01, 2.5684020e-04],
       [2.4284853e-02, 3.0051067e-03, 9.7270995e-01],
       ...,
       [7.7220360e-03, 9.9039090e-01, 1.8870337e-03],
       [2.5963894e-04, 2.4598932e-02, 9.7514141e-01],
       [1.2172571e-07, 1.7309020e-08, 9.9999988e-01]], dtype=float32)

In [69]:
y_true = np.argmax(y_test, axis=1)
pred = np.argmax(pred, axis=1)

print(classification_report(y_true, pred))

              precision    recall  f1-score   support

           0       0.91      0.68      0.78       182
           1       0.85      0.97      0.91       354
           2       0.99      0.98      0.98       227

    accuracy                           0.90       763
   macro avg       0.92      0.87      0.89       763
weighted avg       0.91      0.90      0.90       763

